## Board class

In [56]:
import numpy as np
class board:
    def __init__(self,obsticles,m=18,n=9):
        self.m=m
        self.n=n
        self.pg=[]
        self.obsticles=obsticles
        self.dots=np.ones([m,n])
    def copy(self):
        e=board(self.obsticles.copy(),self.m,self.n)
        e.pg=[i.copy() for i in self.pg]
        for i in range(len(e.pg)):
            e.pg[i].index=i
            e.pg[i].enviroment=e
        e.dots=self.dots.copy()
        return e
    def utility(self):
        gain=self.pg[0].score-self.dots.sum()+self.m*self.n
        return gain
    def add_pg(self,pg):
        self.pg.append(pg)
    def request_move(self,w,index):
        flag=False
        pg=self.pg[index]
        next=np.array([pg.x,pg.y])
        if w==0  and pg.y<self.n-1:
            next=np.array([pg.x,pg.y+1])
        elif w==1  and pg.x<self.m-1:
            next=np.array([pg.x+1,pg.y])
        elif w==2  and pg.y>0:
            next=np.array([pg.x,pg.y-1])
        elif w==3 and pg.x>0 :
            next=np.array([pg.x-1,pg.y])
        if not((self.obsticles==next).all(axis=1)).any():
            if not (next[0],next[1])==(pg.x,pg.y):
                flag=True
            pg.x,pg.y=next[0],next[1]
        if index==0 and flag:
            pg.score=pg.score-1
            if self.dots[next[0],next[1]]==1:
                self.dots[next[0],next[1]]=0
                pg.score=pg.score+10   
    def Map(self):
        b=np.zeros([self.m,self.n],dtype=np.int8)
        for i in self.pg:
            b[i.x,i.y]=2
        b[self.pg[0].x,self.pg[0].y]=1
        b[self.obsticles[:,0],self.obsticles[:,1]]=-1
        return b 
    def is_leaf(self):
        cond=(self.pg[0].x,self.pg[0].y==self.pg[1].x,self.pg[1].y) or (self.pg[0].x,self.pg[0].y==self.pg[2].x,self.pg[2].y) or self.dots.sum()==np.shape(self.obs)[0]
        return cond 
    def step(self):
        for i in range(len(self.pg)):
             self.pg[i].go()          
        

## Class of PG that is parent of of ghost and pacman .

In [57]:
class PG:
    def __init__(self,x,y,enviroment):
        self.x=x
        self.y=y
        self.enviroment=enviroment
        enviroment.add_pg(self)
        self.index=len(enviroment.pg)-1
    def move(self,w):
        self.enviroment.request_move(w,self.index)

## Pacman class

In [58]:
class Pacman(PG):
    max_depth=7
    def __init__(self, x, y, enviroment=None):
        super().__init__(x, y, enviroment)
        self.score=0
        self.enviroment.dots[x,y]=0
        self.enviroment.pg
    def copy(self):
        a =Pacman(self.x,self.y,self.enviroment)
        a.enviroment.pg.pop()
        a.score=self.score
        a.enviroment.dots=self.enviroment.dots.copy()
        return a
    def decsision(self):
        b=self.enviroment
        max_i,i=self.Max_search(b)
        return i
    def Max_search(self,b0,depth=0):
        l=[]
        print(depth)
        if depth<Pacman.max_depth:
            for i in range(4):
                b=b0.copy()
                b.pg[0].move(i)
                l.append(b.pg[0].Min_search(b,depth+1))
            return max(l),np.argmax(l)
        else:
            b=b0.copy()
            return b.utility(),1
    def Min_search(self,b0,depth):
        l=[]
        if depth<Pacman.max_depth and self.enviroment.is_leaf():
            for j in range(4):
                for k in range(4):
                    b=b0.copy()
                    b.pg[1].move(j)
                    b.pg[2].move(k)
                    l.append(b.pg[0].Max_search(b,depth+1)[0])
            return min(l)
        else:
            b=b0.copy()
            return b.utility()
    def name(self):
        return f"pacman"
    def go(self):
        i=self.decsision()
        self.move(i)

## Ghost class

In [59]:
class ghost(PG):
    def __init__(self, x, y, enviroment):
        super().__init__(x, y, enviroment)
    def go(self):
        self.move(np.random.choice(4))
    def name(self):
        return f"ghost{self.index}"
    def copy(self):
        a =ghost(self.x,self.y,self.enviroment)
        a.enviroment.pg.pop()
        return a

## Running of game

In [60]:
from pandas import read_csv as read
r=read('dots.csv')
r=np.array(r)
m=18
n=9
k=4
obs=r[0:56]
b=board(obs,m,n)
p=Pacman(r[55,0],r[55,1],b)
g1=ghost(r[57,0],r[57,1],b)
g2=ghost(r[56,0],r[56,1],b)
for i in range(10):
    b.step()
    for j in b.pg:
        print(f"{j.name()} in steps{i} position at {j.x,j.y}")
print(b.dots)
print(b.pg[0].score)

0
2
4
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
4
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
4
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
4
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
4
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
4
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
4
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
4
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6


KeyboardInterrupt: 